In [2]:
from numpy import sum, sqrt, sort, argsort, zeros, arange, array, isclose
from numpy.random import randint, rand
from subprocess import call
import matplotlib.pyplot as plt

import numpy as np

In [3]:
def write_input(X,Y):
    n,d = X.shape
    m,d = Y.shape
    with open("data.in", 'w') as file:
        # Write the dimetions in the file
        file.write("%d %d %d\n"%(n,m,d))
    
        #Write X in the file
        for row in X:
            for item in row:
                file.write("%g "%item)
            file.write("\n")
    
        #Write X in the file
        for row in Y:
            for item in row:
                file.write("%g "%item)
            file.write("\n")
    return

In [150]:
def run (k):
    call(['./main_sequential', str(k)])

In [4]:
def kNN(X,Y,k):
    # sqrt( sum(X.^2,2) - 2*X*Y' + sum(Y.^2,2)' )
    D = sqrt(
            # sum(X.^2,2) - 2*X*Y' + sum(Y.^2,2)'
            sum(Y**2,axis=1, keepdims=True) - 2 * Y@X.T + sum(X**2,axis=1,keepdims=True).T
            )
    
    m,n   =  D.shape
    nidx  =  zeros((m,k))
    ndist =  zeros((m,k))
    
    for qp in range(m):
        nidx[qp,:]  =  argsort(D[qp,:])[:k]
        ndist[qp,:] =  sort(D[qp,:])[:k]
        
    return nidx, ndist, m, k

In [30]:
def read_output():
    
    
    with open("data.out", "r") as file:
        # Read the dimentions
        first_line = file.readline()
        m,k = list(map(int, first_line.strip().split()))
        
        # Read the indexes
        nidx = zeros((m,k))
        for i in range(m):
            next_line = file.readline()
            nidx[i,:] = list(map(float, next_line.strip().split()))
        
        # Read the distances
        ndist = zeros((m,k))
        for i in range(m):
            next_line = file.readline()
            ndist[i,:] = list(map(float, next_line.strip().split()))
    
    return nidx, ndist, m, k

In [351]:
X = rand(1000, 10)
Y = rand(2, 10)
k = 60

In [352]:
write_input(X,Y)
run(k)

In [353]:
knn = kNN(X,Y,k)

In [354]:
res = read_output()

In [355]:
wf   = np.where(res[0]!=knn[0])
fidx = list(zip(wf[0],wf[1]))

In [356]:
fidx

[(0, 57),
 (0, 58),
 (0, 59),
 (1, 46),
 (1, 47),
 (1, 48),
 (1, 49),
 (1, 50),
 (1, 51),
 (1, 52),
 (1, 53),
 (1, 54),
 (1, 55),
 (1, 56),
 (1, 57),
 (1, 58),
 (1, 59)]

In [365]:
res[1][fidx[4]]

1.016427

In [366]:
knn[1][fidx[4]]

1.0129145135861695